In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import shutil

# Directorios de datos originales
base_dir = "data"
train_dir = os.path.join(base_dir, "train")
valid_dir = os.path.join(base_dir, "valid")

# Directorios para datos procesados
processed_base = "data/processed"
processed_train_dir = os.path.join(processed_base, "train")
processed_valid_dir = os.path.join(processed_base, "validation")

os.makedirs(processed_train_dir, exist_ok=True)
os.makedirs(processed_valid_dir, exist_ok=True)

# Mapeo de clases a identificadores
class_mapping = {
  "fish": 1,
  "jellyfish": 2,
  "penguin": 3,
  "shark": 4,
  "puffin": 5,
  "stingray": 6,
  "starfish": 7
}

def process_folder(input_folder, output_folder, ann_df):
  """
  Procesa las imágenes en 'input_folder' y guarda en 'output_folder':
    - Copia la imagen original.
    - Si la imagen contiene más de una clase, genera una imagen sintética donde se reemplazan
      los píxeles de la clase predominante por el color medio del resto de la imagen.
    - Se registra la información en new_dataset.csv:
        * Para la imagen original se utiliza la primera clase (más frecuente).
        * Para la imagen sintética se utiliza la segunda clase (si existe).
  """
  info = []  # Lista para almacenar la información que se registrará en el CSV

  # Se obtienen los archivos de imagen (.jpg, .jpeg, .png)
  image_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

  for name in tqdm(image_files, desc=f"Procesando {os.path.basename(input_folder)}"):
    path = os.path.join(input_folder, name)
    image = Image.open(path).convert("RGB")
    w, h = image.size
    mask = np.zeros((h, w), dtype=np.uint8)

    # Asignar en la máscara el identificador de cada clase según las anotaciones
    for _, row in ann_df[ann_df["filename"] == name].iterrows():
      cid = class_mapping[row["class"]]
      mask[int(row["ymin"]):int(row["ymax"]), int(row["xmin"]):int(row["xmax"])] = cid

    # Se obtienen las clases presentes y se elimina el fondo (0)
    uniques, counts = np.unique(mask, return_counts=True)
    counts_dict = dict(zip(uniques, counts))
    counts_dict.pop(0, None)  # Se elimina el fondo

    # Se ordenan las clases por cantidad de píxeles (de mayor a menor)
    sorted_classes = sorted(counts_dict.items(), key=lambda x: x[1], reverse=True)

    # Verificar que sorted_classes no esté vacío antes de acceder a su primer elemento
    if not sorted_classes:
      print(f"No se encontraron anotaciones para la imagen: {name}")
      continue  # Salta al siguiente elemento

    prim_id, prim_area = sorted_classes[0]
    prim_label = next(label for label, idx in class_mapping.items() if idx == prim_id)

    # Registro para la imagen original
    info.append({"filename": name, "predominant_class": prim_label, "area": prim_area})
    shutil.copy(path, os.path.join(output_folder, name))

    # Solo se genera la imagen sintética si hay más de una clase anotada
    if len(sorted_classes) > 1:
      sec_id, sec_area = sorted_classes[1]
      sec_label = next(label for label, idx in class_mapping.items() if idx == sec_id)

      arr = np.array(image)
      mask_prim = (mask == prim_id)
      mean_color = arr[~mask_prim].mean(axis=0).astype(np.uint8)
      syn_arr = arr.copy()
      syn_arr[mask_prim] = mean_color

      base, _ = os.path.splitext(name)
      syn_name = f"{base}_synthetic.jpg"
      Image.fromarray(syn_arr).save(os.path.join(output_folder, syn_name))

      info.append({"filename": syn_name, "predominant_class": sec_label, "area": sec_area})

  pd.DataFrame(info).to_csv(os.path.join(output_folder, "new_dataset.csv"), index=False)
  print(f"Procesamiento completado en {os.path.basename(input_folder)}. CSV guardado.")

# Lectura de anotaciones para conjuntos de entrenamiento y validación
train_ann = pd.read_csv(os.path.join(train_dir, "annotations.csv"))
valid_ann = pd.read_csv(os.path.join(valid_dir, "annotations.csv"))

process_folder(train_dir, processed_train_dir, train_ann)
process_folder(valid_dir, processed_valid_dir, valid_ann)

print("Todos los conjuntos han sido procesados correctamente.")

Procesando train: 100%|██████████| 1/1 [00:00<00:00, 59.27it/s]


No se encontraron anotaciones para la imagen: IMG_3133_jpeg_jpg.rf.f439b9d382fd153b96f0a88cdf169172.jpg
Procesamiento completado en train. CSV guardado.


Procesando valid: 0it [00:00, ?it/s]

Procesamiento completado en valid. CSV guardado.
Todos los conjuntos han sido procesados correctamente.


In [ ]:
import tensorflow as tf
import pandas as pd
import os

# Ruta del archivo CSV
csv_path = r'data/processed/train/new_dataset.csv'
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"No se encontró el archivo CSV en: {csv_path}")

# Cargar el CSV usando pandas
df = pd.read_csv(csv_path)
rutas_archivos = df['filename'].tolist()
etiquetas = df['predominant_class'].tolist()

# Obtener el directorio donde se encuentra el CSV y construir la ruta completa de cada imagen
directorio_csv = os.path.dirname(os.path.abspath(csv_path))
print("Directorio CSV:", directorio_csv)
rutas_imagenes = [os.path.join(directorio_csv, nombre) for nombre in rutas_archivos]

# Función auxiliar que se ejecuta en modo eager a través de tf.py_function
def _cargar_imagen(ruta, etiqueta):
    # Convertir el tensor a un valor numpy y luego decodificarlo a str
    ruta_str = ruta.numpy().decode('utf-8')
    # print("Cargando la imagen desde:", ruta_str)

    # Leer y decodificar la imagen
    imagen_bytes = tf.io.read_file(ruta_str)
    imagen_decoded = tf.image.decode_image(imagen_bytes, channels=3, expand_animations=False)
    imagen_resized = tf.image.resize(imagen_decoded, [256, 256])

    return imagen_resized, etiqueta

# Función de mapeo que utiliza tf.py_function para ejecutar _cargar_imagen en modo eager
def cargar_imagen(ruta, etiqueta):
    imagen, etiqueta_out = tf.py_function(
        func=_cargar_imagen,
        inp=[ruta, etiqueta],
        Tout=[tf.float32, etiqueta.dtype]
    )
    # Definir la forma fija de la imagen resultante para el grafo
    imagen.set_shape([256, 256, 3])
    return imagen, etiqueta_out

# Crear el dataset a partir de la lista de rutas y etiquetas
dataset = tf.data.Dataset.from_tensor_slices((rutas_imagenes, etiquetas))
dataset = dataset.map(cargar_imagen, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(buffer_size=1000)
dataset = dataset.batch(1)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

# Ejemplo de iterar sobre el dataset
for imagenes, labels in dataset.take(1):
    print("Batch de imágenes:", imagenes.shape)
    print("Batch de etiquetas:", labels)


EmptyDataError: No columns to parse from file

In [ ]:
ds_iter = dataset.take(1).as_numpy_iterator()
for el in ds_iter:
    print("Imagen:", el[0].shape, "Etiqueta:", el[1])
    break  # Solo para mostrar un ejemplo